In [1]:
#run this command in order to install the packages
#pip install indra
#pip install pybel_jupyter
import indra
import pybel
import pybel_jupyter

INFO: [2024-06-26 14:57:14] numexpr.utils - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: [2024-06-26 14:57:14] numexpr.utils - NumExpr defaulting to 8 threads.


In [2]:
from indra.sources import trips
from indra.statements.concept import Concept
from indra.statements.statements import *
from indra.preassembler import Preassembler
from indra.assemblers.pybel import assembler
from pybel_jupyter import to_jupyter

In [3]:
#gene upregulation related to subtype
class Association(RegulateActivity):
    """Indicates that a gene upregulation is related to a subtype
    
    This statement is intended to be used for a relationship between a upregulation 
    of a gene set. 

    This statement is intended to be used for physical interactions where the
    mechanism of activation is not explicitly specified, which is often the
    case for descriptions of mechanisms extracted from the literature.

    Parameters
    ----------
    subj : :py:class:`Agent`
        The agent responsible for the change in activity, i.e., the "upstream"
        node.
    obj : :py:class:`Agent`
        The agent whose activity is influenced by the subject, i.e., the
        "downstream" node.
    obj_activity : Optional[str]
        The activity of the obj Agent that is affected, e.g., its "kinase"
        activity.
    evidence : None or :py:class:`Evidence` or list of :py:class:`Evidence`
        Evidence objects in support of the modification.

    Examples
    --------

    MEK (MAP2K1) activates the kinase activity of ERK (MAPK1):

    >>> mek = Agent('MAP2K1')
    >>> erk = Agent('MAPK1')
    >>> act = Activation(mek, erk, 'kinase')
    """

    def __init__(self, subj, obj, obj_activity='activity', evidence=None):
        super(RegulateActivity, self).__init__(evidence)
        self.subj = subj
        self.obj = obj
        if obj_activity not in activity_types:
            logger.warning('Invalid activity type: %s' % obj_activity)
        self.obj_activity = obj_activity
        self.is_activation = False


In [4]:
#gene upregulation related to subtype
class clinical_benefit(RegulateActivity):
    """Indicates that a gene upregulation is related to a subtype
    
    This statement is intended to be used for a relationship between a upregulation 
    of a gene set. 

    This statement is intended to be used for physical interactions where the
    mechanism of activation is not explicitly specified, which is often the
    case for descriptions of mechanisms extracted from the literature.

    Parameters
    ----------
    subj : :py:class:`Agent`
        The agent responsible for the change in activity, i.e., the "upstream"
        node.
    obj : :py:class:`Agent`
        The agent whose activity is influenced by the subject, i.e., the
        "downstream" node.
    obj_activity : Optional[str]
        The activity of the obj Agent that is affected, e.g., its "kinase"
        activity.
    evidence : None or :py:class:`Evidence` or list of :py:class:`Evidence`
        Evidence objects in support of the modification.

    Examples
    --------

    MEK (MAP2K1) activates the kinase activity of ERK (MAPK1):

    >>> mek = Agent('MAP2K1')
    >>> erk = Agent('MAPK1')
    >>> act = Activation(mek, erk, 'kinase')
    """

    def __init__(self, subj, obj, obj_activity='activity', evidence=None):
        super(RegulateActivity, self).__init__(evidence)
        self.subj = subj
        self.obj = obj
        if obj_activity not in activity_types:
            logger.warning('Invalid activity type: %s' % obj_activity)
        self.obj_activity = obj_activity
        self.is_activation = False


In [5]:
pMMR_CRC = Agent('pMMR_CRC', db_refs = {'TEXT': 'MSI Immune'})
dMMR_mCRC = Agent('dMMR_mCRC', db_refs ={'TEXT': 'MMR deficient metastasis colorectal cancer'})

recurrence_rate = Agent('recurrence_rate', db_refs = {'TEXT': 'recurrence rate'})

#CMS
CMS1 = Agent('CMS1', db_refs = {'TEXT': 'CMS1'})

#gene ID

#treatment/therapy
adjuvant_chemo = Agent('adjuvant_chem', db_refs = {'TEXT': 'adjuvant chemotherapy'})
Anti_PD1 = Agent('Anti_PD1', db_refs = {'TEXT': 'Anti PD1 therapy'})

#drug
pembrolizumab = Agent('pembrolizumab', db_refs = {'PUBCHEM': '187051801'})
bevacizumab = Agent('bevacizumab', db_refs = {'PUBCHEM': '481101539'})

#literature evidence
evi_21597022 = Evidence(text = '21597022', pmid = 'PMC3110173')
evi_26028255 = Evidence(text ='26028255', pmid ='PMC4481136')
evi_34077519 = Evidence(text ='34077519', pmid ='PMC9002278')
evi_35254400 = Evidence(text ='35254400', pmid = 'PMC9365358')

In [6]:
pbmi_21597022_1 = Association(pMMR_CRC, recurrence_rate, evidence = evi_21597022)
pbmi_21597022_2 = Inhibition(recurrence_rate, adjuvant_chemo, evidence = evi_21597022)
pbmi_26028255_1 = clinical_benefit(dMMR_mCRC, pembrolizumab, evidence = evi_26028255)
pbmi_34077519_1 = clinical_benefit(CMS1, bevacizumab, evidence = evi_34077519)
pbmi_35254400_1 = clinical_benefit(CMS1, Anti_PD1, evidence = evi_35254400)

In [7]:
stmt_all = [pbmi_21597022_1, pbmi_21597022_2, pbmi_26028255_1, pbmi_34077519_1, pbmi_35254400_1]

In [8]:
pba = assembler.PybelAssembler(stmt_all)

In [9]:
#code to give the citation
pba.statements

[Association(pMMR_CRC(), recurrence_rate()),
 Inhibition(recurrence_rate(), adjuvant_chem()),
 clinical_benefit(dMMR_mCRC(), pembrolizumab()),
 clinical_benefit(CMS1(), bevacizumab()),
 clinical_benefit(CMS1(), Anti_PD1())]

In [10]:
#transform the ensemble into pyBEL 
belgraph = pba.make_model()

In [11]:
#visualize using pyBEL jupyter notebook package
belgraph

Name,indra
Version,03cf96d4-ad29-4bbf-a6bf-d9141c999ba4
Nodes,8
Namespaces,2
Edges,5
Annotations,4
Citations,4
Authors,0
Components,3
Warnings,0
Network Density,0.0893


In [12]:
to_jupyter(belgraph)

<IPython.core.display.Javascript object>